<center> <h1> Teoria de Juegos </h1> </center>
<center> <h4> 1er Exámen Parcial (Parte de programación en Python con libreria Mesa) </h4> </center>

<center> <h3> Profesores: Matías Alvarado, Sergio Alcalá, Departamento de Computación, CINVES
 </h3> </center>
 

<center> <h3>  Moises Omar León PIenda </h3> </center>
 


Sean  conjuntosde  agentes que  interactúanen  un  entorno  definido, circular o  poligonal,conforme el Modelo de Ising en 2 dimensiones:

  1.Programar el modelo de Ising, con spines (agentes)tales que: (1 puntos)

    a.Los   conjuntosA1   y   A2 de   cardinalidad nym,cooperan entre   sí, respectivamente. 
    b.A1 y A2 compiten entre sí, conforme el modelo de Ising. 
    c.Entradas de cada conjunto dadaporfunción de distribución uniforme.
    d.Hay factores externos μ1  y + μ2 cuyos  eventos  ocurren conforme  una distribución normal.


  2.Con entradas  dadas  por  unadistribución normal  de  probabilidades.Interpretar tal que:(2.5puntos)
  
    a.A1 es cáncer, C.
    b.A2 es sistema inmune, SI.
    c.El factor μ 1es favorable adesarrollo de C, y un μ2 favorableal SI.
    

  3.Simulaciones:definir y diseñar 3 escenarios:(2.5puntos)
  
    a.A1 + μ1 gana o escapa a A2+μ2.
    b.A1 + μ1 y A2 + μ2 están en equilibrio.
    c.A2+ μ2 gana o elimina a A1 + μ1

# Solución 




El modelo de Ising es un modelo matemático simple que permite estudiar un sistema de spines (o dipolos magneticos) interactuantes. El modelo en 2 dimensiones da cuenta de un cambio de fase a baja temperatura, en la cual el sistema se magnetiza. El Hamiltoniano que describe el sistema es 

 H = - ∑<sub>ij</sub>J<sub>ij</sub>S<sub>i</sub>S<sub>j - </sub>μ ∑<sub>j </sub>h <sub> j </sub> S <sub> j


La interacción spin-spin se clasifica de la siguiente manera:

-J<sub>ij</sub> >0: ferromagnética

-J<sub>ij</sub> <0: antiferromagnética

-J<sub>ij </sub> =0: no interactuante 


La versión más simple del modelo de Ising considera interacción únicamente a primeros vecinos, sin la presencia den un campo magnético externo. El Hamiltoniano en este caso es

H = -J ∑<sub>ij </sub>S<sub>i</sub>S<sub>j

donde J=J<sub>ij </sub> toma el mismo valor para todos los pares de spines.     
 


# Primero importamos las librerias que no se encuentran en colab

Para importar una biblioteca que no está en Colaboratory de forma predeterminada, puede utilizar

`!pip install` or `!apt-get install`.

In [ ]:
#Matplotlib
!pip install matplotlib-venn

#Mesa
!pip install mesa

from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
import numpy as np
import networkx as nx
import random


     |████████████████████████████████| 691 kB 4.2 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 


In [11]:
def f_difference(model):
    diff = 0
    conn = nx.edges(model.graph)
    for edge in conn:
        diff += np.mean(abs(model.schedule.agents[edge[0]].feature - model.schedule.agents[edge[1]].feature))
    return diff

In [10]:


class ising_Agent(Agent):
    ## Individual agent represented by a node in a network

    ## Initializing the agent.
    def __init__(self, unique_id, agent_type, model):
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.type = agent_type
        
    
    ## No content on agent's step.
    def step(self):
      #Hamintoniano

        return
    
class ising_model(Model):
    #Model containing N agents connected in a network. Also change the value of agent here
    
    ##Initializing the model.
    def __init__(self, height=20, width=20, minority_pc=0.5, T=0.4, J=0.0, normal=False, 
    media1 = 0.5, desviacion1=0.5, media2 = 0.5, desviacion2=0.5):
        """ """
        # atributos del modelo 
        self.height = height
        self.width = width
        self.minority_pc = minority_pc       
        self.T_inv = 1/T
        self.total_energy =0
        self.J =J
        self.normal=normal
        self.media1 = media1
        self.desviacion1= desviacion1
        self.media2 = media2
        self.desviacion2= desviacion2

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(width, height, torus=True)

        self.datacollector = DataCollector(
            {"energy": "energy"}, 
            {"x": lambda a: a.unique_id[0], "y": lambda a: a.unique_id[1]},
        )
        
        ## Creating the agents.
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            if self.random.random() < self.minority_pc:
                agent_type = 1              # células cancerosas
            else:
                agent_type = -1             # SI

            agent = IsingAgent((x, y), self, agent_type)
            self.grid.unique_idition_agent(agent, (x, y))
            self.schedule.add(agent)
        for cell in self.grid.coord_iter():
            # Energía total
            spin = cell[0].type
            x = cell[1]
            y = cell[2]
            energia_inicial =0
            for neighbor in self.grid.neighbor_iter((x,y), False): 
                energia_inicial += neighbor.type
            self.total_energy += -energia_inicial*spin/2


        ## Creating collector for difference function defined above.
        self.running = True
        self.datacollector.collect(self)
    
    ## At each model step, they collect data by using the collector.
    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)

In [12]:


from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.UserParam import UserSettableParameter



class Element(TextElement):
#Display a text count of how many happy agents there are.
    


    def __init__(self):
        pass

    def render(self, model):
        return "Energy agents: " + str(model.total_energy )


def schelling_draw(agent):
    """
    Portrayal Method for canvas
    """
    if agent is None:
        return
    portrayal = {"Shape": "rect", "w":1, "h": 1, "Filled": "true", "Layer": 0}

    if agent.type == 1:
        portrayal["Color"] = ["#FF0000", "#FF9999"]
        portrayal["stroke_color"] = "#00FF00"
    else:
        portrayal["Color"] = ["#0000FF", "#9999FF"]
        portrayal["stroke_color"] = "#000000"
    return portrayal

energy_element = Element()

height = 50
width = 50 
canvas_element = CanvasGrid(schelling_draw, height, width, 500, 500)
e_chart = ChartModule([{"Label": "total_energy", "Color": "Black"}])

model_params = {
    "height": height,
    "width": width,
    "minority_pc": UserSettableParameter("slider", "Porcentaje de azules", 0.5, 0.01, 1.0, 0.005),
    "T": UserSettableParameter("slider", "T (temperatura)", 0.4, 0.0, 5, 0.1),
    "J": UserSettableParameter("slider", "J", -0.5, -2, 2, 0.01),
    "normal": UserSettableParameter("checkbox", "Distribución normal de mu1 y mu2", value=False),
    "media1": UserSettableParameter("number", "Media mu1", value=0.5),
    "desviacion1": UserSettableParameter("number", "Desviación estandar mu1", value=0.5),
    "media2": UserSettableParameter("number", "Media mu2", value=0.5),
    "desviacion2": UserSettableParameter("number", "Desviación estandar m2", value=0.5),

}
 
